## 作業
礙於不是所有同學都有 GPU ，這邊的範例使用的是簡化版本的 ResNet，確保所有同學都能夠順利訓練!


最後一天的作業請閱讀這篇非常詳盡的[文章](https://blog.gtwang.org/programming/keras-resnet-50-pre-trained-model-build-dogs-cats-image-classification-system/)，基本上已經涵蓋了所有訓練　CNN 常用的技巧，請使用所有學過的訓練技巧，盡可能地提高 Cifar-10 的 test data 準確率，截圖你最佳的結果並上傳來完成最後一次的作業吧!

另外這些技巧在 Kaggle 上也會被許多人使用，更有人會開發一些新的技巧，例如使把預訓練在 ImageNet 上的模型當成 feature extractor 後，再拿擷取出的特徵重新訓練新的模型，這些技巧再進階的課程我們會在提到，有興趣的同學也可以[參考](https://www.kaggle.com/insaff/img-feature-extraction-with-pretrained-resnet)

In [1]:
import keras
from keras.layers import Dense, Conv2D, BatchNormalization, Activation
from keras.layers import AveragePooling2D, Input, Flatten
from keras.optimizers import Adam
from keras.regularizers import l2
from keras import backend as K
from keras.models import Model
from keras.datasets import cifar10
from keras.callbacks import ModelCheckpoint, LearningRateScheduler
from keras.callbacks import ReduceLROnPlateau
import numpy as np
import os


In [40]:
pip install --upgrade tensorflow-gpu --user


Requirement already up-to-date: tensorflow-gpu in d:\anaconda3\lib\site-packages (2.5.0)
Note: you may need to restart the kernel to use updated packages.


In [2]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())

[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13943869600124854922
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 4848943104
locality {
  bus_id: 1
  links {
  }
}
incarnation: 16492749933872940307
physical_device_desc: "device: 0, name: NVIDIA GeForce GTX 1060, pci bus id: 0000:01:00.0, compute capability: 6.1"
]


In [4]:
(x_train,y_train),(x_test,y_test) = cifar10.load_data()

In [5]:
x_train = x_train/255
x_test = x_test/255
y_train = keras.utils.to_categorical(y_train,10)
y_test = keras.utils.to_categorical(y_test,10)

In [6]:
def resnet_block(inputs,num_filters=16,
                  kernel_size=3,strides=1,
                  activation='relu'):
    x = Conv2D(num_filters,kernel_size=kernel_size,strides=strides,padding='same',
           kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(inputs)
    x = BatchNormalization()(x)
    if(activation):
        x = Activation('relu')(x)
    return x

In [7]:
def resnet_v1(input_shape):
    inputs = Input(shape=input_shape)# Input层，用来当做占位使用
    
    #第一层
    x = resnet_block(inputs)
    print('layer1,xshape:',x.shape)
    # 第2~7层
    for i in range(6):
        a = resnet_block(inputs = x)
        b = resnet_block(inputs=a,activation=None)
        x = keras.layers.add([x,b])
        x = Activation('relu')(x)
    # out：32*32*16
    # 第8~13层
    for i in range(6):
        if i == 0:
            a = resnet_block(inputs = x,strides=2,num_filters=32)
        else:
            a = resnet_block(inputs = x,num_filters=32)
        b = resnet_block(inputs=a,activation=None,num_filters=32)
        if i==0:
            x = Conv2D(32,kernel_size=3,strides=2,padding='same',
                       kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(x)
        x = keras.layers.add([x,b])
        x = Activation('relu')(x)
    # out:16*16*32
    # 第14~19层
    for i in range(6):
        if i ==0 :
            a = resnet_block(inputs = x,strides=2,num_filters=64)
        else:
            a = resnet_block(inputs = x,num_filters=64)

        b = resnet_block(inputs=a,activation=None,num_filters=64)
        if i == 0:
            x = Conv2D(64,kernel_size=3,strides=2,padding='same',
                       kernel_initializer='he_normal',kernel_regularizer=l2(1e-4))(x)
        x = keras.layers.add([x,b])# 相加操作，要求x、b shape完全一致
        x = Activation('relu')(x)
    # out:8*8*64
    # 第20层   
    x = AveragePooling2D(pool_size=2)(x)
    # out:4*4*64
    y = Flatten()(x)
    # out:1024
    outputs = Dense(10,activation='softmax',
                    kernel_initializer='he_normal')(y)
    
    #初始化模型
    #之前的操作只是将多个神经网络层进行了相连，通过下面这一句的初始化操作，才算真正完成了一个模型的结构初始化
    model = Model(inputs=inputs,outputs=outputs)
    return model

model = resnet_v1((32,32,3))

model.compile(loss='categorical_crossentropy',
optimizer=Adam(),
metrics=['accuracy'])

model.summary()


layer1,xshape: (None, 32, 32, 16)
Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 32, 32, 16)   448         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 32, 32, 16)   64          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 32, 32, 16)   0           batch_normalization[0][0]        
____________________________________________________________

In [17]:
checkpoint = ModelCheckpoint(filepath='./cifar10_resnet_ckpt.h5',monitor='val_accuracy',
                             verbose=1,save_best_only=True)
def lr_sch(epoch):
    #200 total
    if epoch <50:
        return 1e-3
    if 50<=epoch<100:
        return 1e-4
    if epoch>=100:
        return 1e-5
lr_scheduler = LearningRateScheduler(lr_sch)
lr_reducer = ReduceLROnPlateau(monitor='val_accuracy',factor=0.2,patience=5,
                               mode='max',min_lr=1e-3)
callbacks = [checkpoint,lr_scheduler,lr_reducer]


In [18]:
import tensorflow as tf
batch_size = 128
epochs = 100
from keras.preprocessing.image import ImageDataGenerator
data_generator = ImageDataGenerator(
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True)
augmented_image = data_generator.flow(x_train,y_train,batch_size = batch_size)
with tf.device('/device:GPU:0'):
                    model.fit_generator(augmented_image,
                   steps_per_epoch = int(len(x_train)/batch_size),
                   verbose = 1,
                   validation_data = (x_test,y_test),
                   epochs = epochs,
                   callbacks = callbacks)
                    
                    

Epoch 1/100
390/390 [==============================] - 40s 101ms/step - loss: 1.1869 - accuracy: 0.6632 - val_loss: 1.4824 - val_accuracy: 0.6026

Epoch 00001: val_accuracy improved from -inf to 0.60260, saving model to .\cifar10_resnet_ckpt.h5


D:\Anaconda3\lib\site-packages\tensorflow\python\keras\utils\generic_utils.py:497: CustomMaskWarning: Custom mask layers require a config and must override get_config. When loading, the custom mask layer must be passed to the custom_objects argument.
  category=CustomMaskWarning)


Epoch 2/100
390/390 [==============================] - 40s 102ms/step - loss: 1.1217 - accuracy: 0.6804 - val_loss: 1.5873 - val_accuracy: 0.5960

Epoch 00002: val_accuracy did not improve from 0.60260
Epoch 3/100
390/390 [==============================] - 40s 103ms/step - loss: 1.0815 - accuracy: 0.6947 - val_loss: 1.7139 - val_accuracy: 0.5624

Epoch 00003: val_accuracy did not improve from 0.60260
Epoch 4/100
390/390 [==============================] - 40s 102ms/step - loss: 1.0229 - accuracy: 0.7143 - val_loss: 1.0121 - val_accuracy: 0.7241

Epoch 00004: val_accuracy improved from 0.60260 to 0.72410, saving model to .\cifar10_resnet_ckpt.h5
Epoch 5/100
390/390 [==============================] - 40s 102ms/step - loss: 0.9827 - accuracy: 0.7295 - val_loss: 1.3500 - val_accuracy: 0.6441

Epoch 00005: val_accuracy did not improve from 0.72410
Epoch 6/100
390/390 [==============================] - 40s 102ms/step - loss: 0.9393 - accuracy: 0.7419 - val_loss: 1.0425 - val_accuracy: 0.7206


In [21]:
score = model.evaluate(x_test,y_test,verbose = 1)
print('Test loss:',score[0])
print('Test accuracy:',score[1])

313/313 [==============================] - 5s 15ms/step - loss: 0.4961 - accuracy: 0.9040
Test loss: 0.4960649311542511
Test accuracy: 0.9039999842643738
